In [1]:
#2017/4/15 完成訓練模組
#考慮自由中國非文藝類，投稿數量前10名的作者
#實驗流程：獲得內文 -> 文本特徵向量建立 -> SMOTE平衡文本量 
# -> GridSearchCV找尋最佳參數並交叉驗證 -> 建立預測報表/點陣圖/預測模型
#特徵採用詞性組合、情態詞+V組合、否定&程度組合
#一般常用的特徵有高頻詞、bigram&trigram、標點符號、豐富度
import codecs
import os
from collections import OrderedDict,defaultdict,Counter
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from matplotlib.font_manager import FontProperties

condicate_author = ['雷震','殷海光','夏道平','龍平甫','蔣勻田','傅正','朱伴耘','胡適','羅鴻詔','聶華苓']

condicate_author_path = "D:\\課業相關\\論文資料\\SVM\\condicate_author\\" #候選作者文本
normal_feature_path = "D:\\課業相關\\論文資料\\SVM\\language_feature\\" #常用特徵
paper_feature_path = "D:\\課業相關\論文資料\\SVM\\language_feature\\二版\\" #論文提出特徵
unknow_author_path = "D:\\課業相關\\論文資料\\SVM\\unknow_author\\" #未知作者文本

print ("library and path OK")

library and path OK


In [2]:
#建立作者索引，提取各作者文章內容及索引
author_index = defaultdict(int)
for index,name in enumerate(condicate_author): #建立作者索引
    author_index[name] = index

author_index = OrderedDict(sorted(author_index.items(), key=lambda t: t[1])) #作者索引排序(依文本數量高到低)

for name,index in author_index.items():
    print (name+':'+str(index),end=' ')
print ()
    
content_list = [] #所有作者文本內容(未處理)
article_label = [] #各文本label，也就是作者索引

for file in os.listdir(condicate_author_path):
    with codecs.open(condicate_author_path+file,'rb','utf8') as f:
        title = f.readline()
        content_list.append(f.readline().strip())
        
        article_label.append(author_index[file.split('_')[0]])
        
print ('共有'+str(len(content_list))+'篇文章')

雷震:0 殷海光:1 夏道平:2 龍平甫:3 蔣勻田:4 傅正:5 朱伴耘:6 胡適:7 羅鴻詔:8 聶華苓:9 
共有429篇文章


In [4]:
#將文章轉換為特徵向量，並平衡作者文本數量(以最多為基準)
feature = [] #特徵
feature_file = '平衡語料庫前100高頻詞.csv'
select_path = normal_feature_path
with codecs.open(select_path+feature_file,'rb','utf8') as f: #抓取基準特徵
    for i in f.readlines():
        if i.strip() != '':
            feature.append(i.strip().split(',')[0])

def line_vec(line): #將文章轉換為特徵向量並回傳
    temp_feature = defaultdict(int)
    
    line = [line[i].split('(')[0] for i in range(len(line))] #詞頻
    #line = [line[i].split('(')[0]+line[i+1].split('(')[0] for i in range(len(line)-1)] #bigram
    #line = [line[i].split('(')[0]+line[i+1].split('(')[0]+line[i+2].split('(')[0] for i in range(len(line)-2)] #trigram
    #line = [line[i]+line[i+1] for i in range(len(line)-1)] #詞性組合
    
    for i in line:
        if i in feature:
            temp_feature[i] += 1
    
    return temp_feature

aa_feature = np.zeros((len(content_list),len(feature)),np.float64)
muti_author = defaultdict(list) #將向量分類

for index,element in enumerate(content_list): #依序將文章轉換為特徵向量
    line = element.split()
    temp_feature = line_vec(line)
    
    for i,j in enumerate(feature):
        aa_feature[index, i] = round(temp_feature[j] * 1000000 / len(line),3) #取相對頻率
        
    muti_author[article_label[index]].append(list(aa_feature[index]))

for index in range(1,len(muti_author)): #將分類後的向量內容數量平均，也就是文本數量一致

    min_target = -1
    min_num = 99
    
    for i,e in muti_author.items():
        if len(e) <= min_num:
            min_num = len(e)
            min_target = i
            
    if len(muti_author[0]) == min_num: #取到文本數量與最大等數值為止，預設最大是第0個
        break
    
    X_t = muti_author[0]+muti_author[min_target]
    y_t = [0]*len(muti_author[0])+[min_target]*len(muti_author[min_target])
    
    X_t, y_t = SMOTE(random_state=0).fit_sample(X_t,y_t) #SMOTE
    muti_author[min_target] = X_t[len(muti_author[0]):]
    
X = [] #處理後文本向量
y = [] #處理後文本label

for k,v in muti_author.items():
    for l in v:
        X.append(l)
    y += [k]*len(v)
    
X = np.array(X)
y = np.array(y)

print ('X:',X.shape)
print ('y:',y.shape)
print ('資料準備已完成')

zero_count = []
for i in range(len(feature)):
    zero_count.append(Counter(X[:,i].tolist())[0]) #找出特徵中0的出現次數
print (zero_count)

X: (860, 100)
y: (860,)
資料準備已完成
[0, 4, 4, 3, 11, 29, 35, 184, 13, 46, 14, 96, 96, 32, 35, 39, 49, 39, 75, 19, 39, 75, 97, 41, 48, 65, 49, 217, 104, 56, 454, 164, 676, 248, 60, 54, 88, 121, 271, 86, 90, 122, 150, 88, 229, 165, 167, 90, 24, 85, 72, 205, 114, 187, 137, 90, 83, 847, 77, 136, 112, 152, 73, 108, 372, 115, 335, 147, 245, 196, 83, 155, 266, 162, 137, 84, 102, 669, 201, 425, 183, 250, 205, 224, 49, 149, 83, 321, 138, 335, 237, 341, 336, 215, 363, 236, 151, 235, 222, 623]


In [ ]:
#使用SVM預測
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0) #分割訓練與測試資料

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}] #候選參數

clf = GridSearchCV(svm.SVC(C=1,probability=True), tuned_parameters, cv=10) #使用交叉驗證並找出最佳參數
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_) #印出最佳參數
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred)) #顯示報表(pred可能和predict_proba不一致，這邊假設為一致)
print()

In [ ]:
#未知作者預測
print (' '.join([str(i)+'.'+e for i,e in enumerate(condicate_author)])) #印出作者
print ()
 
print ('最佳預測精準度：',clf.best_score_)
print ()

for file in os.listdir(unknow_author_path):
    pred_y = np.zeros((1,len(feature)),np.float64)
    with codecs.open(unknow_author_path+file,'rb','utf8') as f:
        head = f.readline().strip()
        content = f.readline().strip().split()

        temp_feature = line_vec(content)

        for i,j in enumerate(feature):
            pred_y[0][i] = round(temp_feature[j] * 1000000 / len(content),3)

    print (head)
    print ('預測作者：'+str(clf.predict(pred_y)[0])+'.'+condicate_author[clf.predict(pred_y)[0]])
    print ([round(i,3) for i in clf.predict_proba(pred_y)[0].tolist()])
    print ()

In [ ]:
#畫出點陣圖
X_reduced = PCA(n_components=2).fit_transform(X) #使用PCA降維

colors = ['black', 'blue', 'purple', 'yellow', 'white', 'red', 'lime', 'cyan', 'orange', 'gray']
for i in range(len(colors)):
    '''if i in [1,3,5,7,9]:
        continue'''
    x_p = X_reduced[:, 0][y == i]
    y_p = X_reduced[:, 1][y == i]
    plt.scatter(x_p, y_p, c=colors[i])
plt.legend(condicate_author, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., \
           prop = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14) ) #顯示中文
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.title("PCA Scatter Plot")
plt.show()

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf, 'C:\\Users\\user\\Desktop\\預測資料\\自bigram.pkl') 
n = joblib.load('C:\\Users\\user\\Desktop\\預測資料\\自bigram.pkl') 
print (n.best_score_)